# 1.1 Amazon SageMaker Training 실습

## 학습 작업의 실행 노트북 개요

이 노트북은 SageMaker에서 Training Job을 통해서 모델 학습합니다.상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 학습](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script)

- 일반적으로 SageMaker Training은 아래와 같이 진행이 됩니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)

### 작업 실행 시 필요 라이브러리 import

In [1]:
%%bash
#!/bin/bash
https://test-instance-250512.notebook.us-east-1.sagemaker.aws/lab/tree/simple-sagemaker/lab_1_training/1.1.SageMaker-Training.ipynb#%ED%95%99%EC%8A%B5-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-%EC%A7%80%EC%A0%95
DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    sudo service docker stop
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo rsync -aP /var/lib/docker /home/ec2-user/SageMaker/.container
    sudo service docker start
    echo "Docker Restart"
fi

Redirecting to /bin/systemctl stop docker.service
  docker.socket


Add data-root and default-shm-size=10G
sending incremental file list
created directory /home/ec2-user/SageMaker/.container
docker/
docker/engine-id
             36 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=36/38)
docker/buildkit/
docker/buildkit/cache.db
         32,768 100%   31.25MB/s    0:00:00 (xfr#2, to-chk=25/38)
docker/buildkit/containerdmeta.db
         16,384 100%   15.62MB/s    0:00:00 (xfr#3, to-chk=24/38)
docker/buildkit/history.db
         32,768 100%   31.25MB/s    0:00:00 (xfr#4, to-chk=23/38)
docker/buildkit/metadata_v2.db
         16,384 100%   15.62MB/s    0:00:00 (xfr#5, to-chk=22/38)
docker/buildkit/snapshots.db
         16,384 100%   15.62MB/s    0:00:00 (xfr#6, to-chk=21/38)
docker/buildkit/content/
docker/buildkit/content/ingest/
docker/buildkit/executor/
docker/containers/
docker/image/
docker/image/overlay2/
docker/image/overlay2/repositories.json
             19 100%   18.55kB/s    0:00:00 (xfr#7, to-chk=16/38)
docker/image/overlay2/distribution/
docker/image

Redirecting to /bin/systemctl start docker.service


Docker Restart


In [3]:
import boto3
import sagemaker

### SageMaker 세션과 Role, 사용 버킷 정의

In [4]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

[05/28/25 02:41:37] INFO     Found credentials from IAM Role:                                   ]8;id=753558;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=101815;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/28/25 02:41:38] INFO     Found credentials from IAM Role:                                   ]8;id=188030;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=770124;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [5]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code'
output_path = f's3://{bucket}/xgboost/output'

### 하이퍼파라미터 정의

In [6]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 실행 작업 정의

In [7]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = "local"
max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [8]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

[05/28/25 02:41:39] INFO     Found credentials from IAM Role:                                   ]8;id=167341;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=399861;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [14]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_script.py",
    source_dir='src',
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.7-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    max_wait=max_wait,
)

[05/28/25 03:28:17] INFO     Ignoring unnecessary Python version: py3.                            ]8;id=596111;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=780198;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#603\603]8;;\

                    INFO     Ignoring unnecessary instance type: ml.m5.large.                     ]8;id=25633;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=796661;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

### 학습 데이터셋 지정

In [15]:
data_path=f's3://{bucket}/xgboost/dataset'
!aws s3 sync ../data/dataset/ $data_path

In [16]:
if instance_type in ['local', 'local_gpu']:
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('lab_1_training')[0] + 'data/dataset/'
    
else:
    inputs = data_path
inputs

's3://sagemaker-us-east-1-992382423459/xgboost/dataset'

### 학습 실행

In [17]:
estimator.fit(inputs = {'inputdata': inputs},
                  wait=False)

[05/28/25 03:28:18] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=579278;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=981783;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=796624;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=370993;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-05-28-03-28-18-601                                             

In [18]:
estimator.logs()

2025-05-28 03:28:20 Starting - Starting the training job...
2025-05-28 03:28:35 Starting - Preparing the instances for training...
2025-05-28 03:28:59 Downloading - Downloading input data...
2025-05-28 03:29:45 Downloading - Downloading the training image.......[2025-05-28 03:30:57.697 ip-10-0-97-244.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-05-28 03:30:57.730 ip-10-0-97-244.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-05-28:03:30:58:INFO] Imported framework sagemaker_xgboost_container.training
[2025-05-28:03:30:58:INFO] No GPUs detected (normal if no gpus installed)
[2025-05-28:03:30:58:INFO] Invoking user training script.
[2025-05-28:03:30:58:INFO] Module xgboost_script_mlflow does not provide a setup.py. 
Generating setup.py
[2025-05-28:03:30:58:INFO] Generating setup.cfg
[2025-05-28:03:30:58:INFO] Generating MANIFEST.in
[2025-05-28:03:30:58:INFO] Installing module with the following command:
/miniconda3/bin/py

### 데이터 세트 설명
- 데이터 세트는 블로그 [Architect and build the full machine learning lifecycle with AWS: An end-to-end Amazon SageMaker demo](https://aws.amazon.com/blogs/machine-learning/architect-and-build-the-full-machine-learning-lifecycle-with-amazon-sagemaker/) 에서 사용한 데이터를 사용합니다. 블로그에서는 데이터 세트에 대해서 이렇게 설명 합니다.
- "자동차 보험 청구 사기를 탐지를 위해서 블로그 저자가 데이터를 합성해서 만든 고객과 클레임의 데이터 세트를 사용합니다."

In [37]:
model_dir = f"{estimator.output_path}/{estimator.latest_training_job.job_name}/output/model.tar.gz"
model_dir

's3://sagemaker-us-east-1-992382423459/xgboost/output/sagemaker-xgboost-2025-05-28-03-28-18-601/output/model.tar.gz'

In [19]:
import pandas as pd

In [20]:
train_prep_df = pd.read_csv('../data/dataset/train.csv')
train_prep_df.groupby('fraud').sample(n=5)

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
486,0,18809.838387,33909.838387,34,5,1,3,750,3000,97623,...,0,0,1,0,0,0,1,0,1,0
3123,0,17421.423480,25521.423480,54,83,0,1,750,3000,85339,...,0,1,0,0,0,0,0,1,0,1
631,0,9727.862240,57327.862240,63,334,0,1,1050,2550,95688,...,0,0,1,0,0,0,1,0,1,0
29,0,14337.538813,22137.538813,48,12,0,1,750,3000,95356,...,0,1,0,0,0,0,0,1,0,1
3699,0,13028.841180,18628.841180,59,57,0,1,750,3000,92867,...,0,0,1,0,0,0,1,0,1,0
1944,1,11361.251613,25861.251613,25,26,0,4,750,3000,94539,...,0,0,1,0,0,0,1,0,1,0
2497,1,3702.585440,17302.585440,28,66,1,1,750,3000,99004,...,0,1,0,0,0,0,0,1,0,1
1688,1,5481.401810,14381.401810,32,66,0,1,750,3000,89801,...,0,0,1,0,0,0,1,0,1,0
101,1,3492.972634,19392.972634,33,24,0,2,750,3000,92131,...,0,0,1,0,0,0,1,0,1,0
908,1,24580.246415,40380.246415,47,24,1,3,750,3000,95337,...,1,1,0,0,0,0,0,1,1,0


In [21]:
train_prep_df.groupby('fraud').size()

fraud
0    3869
1     131
dtype: int64

In [38]:
%store model_dir

Stored 'model_dir' (str)


#### 데이터 컬럼 설명
- fraud: 보험 청구의 사기 여부 입니다. 1 이면 사기, 0 이면 정상 청구 입니다.
- vehicle_claim: 자동차에 대한 보험 청구액. 값으로서, $1000, $17,638 등이 있습니다.
- total_claim_amount: 전체 보험 청구액 입니다. $21,400, $10,000 등이 있습니다.    
- customer_age: 고객의 나이를 의미 합니다.
- months_as_customer: 고객으로서의 가입 기간을 의미합니다. 단위는 월로서 11, 30, 31 등의 값이 존재 합니다.
- num_claims_past_year: 작년의 보험 청구 수를 의미 합니다. 0, 1, 2, 3, 4, 5, 6 의 값이 존재 합니다.
- num_insurers_past_5_years: 과거 5년 동안의 보험 가입 회사 수를 의미 합니다. 1, 2, 3, 4, 5 의 값이 존재 합니다.
- policy_deductable: 보험의 최소 자기 부담금 입니다. $750, $800 등이 있습니다.    
- policy_annual_premium: 보험의 특약 가입에 대한 금액 입니다. $2000, $3000 등이 있습니다.
- customer_zip: 고객의 집 주소 우편 번호를 의미합니다.
- auto_year: 자동차의 년식을 의미 합니다. 2020, 2019 등이 있습니다.
- num_vehicles_involved: 몇 대의 자동차가 사고에 연관 되었는지 입니다. 1, 2, 3, 4, 5, 6 의 값이 있습니다.
- num_injuries: 몇 명이 상해를 입었는지를 기술합니다. 0, 1, 2, 3, 4, 의 값이 있습니다.
- num_witnesses: 몇 명의 목격자가 있었는지를 기술합니다. 0, 1, 2, 3, 4, 5 의 값이 있습니다.
- injury_claim: 상해에 대한 보험 청구액. \$5,500, \$70,700, \$100,700 등이 있습니다.    
- incident_month: 사고가 발생한 월을 의미합니다. 1~12 값이 존재 합니다.
- incident_day: 사고가 발생한 일자를 의미합니다. 1~31 값이 존재 합니다.
- incident_dow: 사고가 발생한 요일을 의미합니다. 0~6 값이 존재 합니다.
- incident_hour: 사고가 발생한 시간을 의미합니다. 0~23 값이 존재 합니다.
- policy_state: 보험 계약을 한 미국 주(State)를 의미 합니다. CA, WA, AZ, OR, NV, ID 가 존재 합니다.    
- policy_liability: 보험 청구의 한도를 의미 합니다. 예를 들어서 25/50 은  사람 당 상해 한도 $25,000, 사고 당 상해 한도가 $50,000 을  의미합니다. 25/50, 15/30, 30/60, 100/200 의 값이 존재 합니다. 
- customer_gender: 고객의 성별을 의미 합니다. Male, Female, Unkown, Other가 존재 합니다.
- customer_education: 고객의 최종 학력을 의미합니다. Bachelor, High School, Advanced Degree, Associate, Below High School 이 존재 합니다.
- driver_relationship: 보험 계약자와 운전자와의 관계 입니다. Self, Spouse, Child, Other 값이 존재 합니다.
- incident_type: 사고의 종류를 기술합니다. Collision, Break-in, Theft 값이 존재 합니다.
- collision_type: 충돌 타입을 기술합니다. Front, Rear, Side, missing 값이 존재 합니다.
- incident_severity: 사고의 손실 정도 입니다. Minor, Major, Totaled 값이 존재 합니다.
- authorities_contacted: 어떤 관련 기관에 연락을 했는지 입니다. Police, Ambuylance, Fire, None 값이 존재 합니다.
- police_report_available: 경찰 보고서가 존재하는지를 기술합니다. Yes, No 의 값이 있습니다.